In [75]:
import numpy as np
from sentence_transformers import SentenceTransformer
import umap
from bokeh.io import curdoc

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LabelSet, CustomJS, Slider, Div
from bokeh.transform import linear_cmap
from bokeh.layouts import column, row
import torch
import scipy.sparse
import numpy as np
output_notebook()

Loading BokehJS ...

In [13]:
ind_to_question = torch.load("./feature_viz/ind_to_question.pt")

rand_activations_sparse = scipy.sparse.load_npz("./feature_viz/rand_activations_sparse.npz")
feature_activations = rand_activations_sparse.toarray()

feature_labels = torch.load("./feature_viz/feature_labels.pt")

/var/folders/9b/9b5ms4js2zn55yhp7yrxlnvm0000gn/T/ipykernel_40898/477134603.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ind_to_question = torch.load("./feature_viz/in

In [18]:
nonzero_features = (feature_activations[:, :max(feature_labels.keys())] > 0).sum(axis = 0)
top_features = np.argsort(nonzero_features)[::-1]
top_features

array([52426, 41637, 37271, ..., 52906, 52905,     0])

In [80]:
np.argsort(feature_activations[:, 5])[::-1]


array([264, 260, 646, 561, 279, 154,  23, 668, 795, 423, 634, 178, 547,
       516, 431, 475, 138, 207,   2, 743, 517, 476, 277, 216, 315, 320,
       323, 322, 321, 308, 307, 316, 309, 319, 310, 311, 312, 318, 313,
       314, 317, 327, 324, 339, 351, 350, 349, 348, 347, 346, 345, 344,
       343, 342, 341, 340, 338, 325, 337, 336, 335, 334, 333, 332, 331,
       330, 329, 328, 305, 326, 306, 297, 304, 303, 272, 271, 270, 269,
       268, 267, 266, 265, 263, 262, 261, 259, 258, 257, 256, 255, 254,
       253, 252, 251, 250, 249, 248, 247, 246, 273, 274, 275, 291, 302,
       301, 300, 299, 298, 353, 296, 295, 294, 293, 292, 290, 276, 289,
       288, 287, 286, 285, 284, 283, 282, 281, 280, 278, 352, 358, 354,
       429, 442, 441, 440, 439, 438, 437, 436, 435, 434, 433, 432, 430,
       428, 444, 427, 426, 425, 424, 422, 421, 420, 419, 418, 417, 416,
       415, 443, 445, 355, 460, 472, 471, 470, 469, 468, 467, 466, 465,
       464, 463, 462, 461, 459, 446, 458, 457, 456, 455, 454, 45

In [99]:
texts = []
numbers = []
expansions_map = {}

for i in top_features[100:]:
  if i in feature_labels and feature_labels[i] not in texts and nonzero_features[i] > 5:
    texts.append(feature_labels[i])
    numbers.append(nonzero_features[i])
    expansions_map[feature_labels[i]] = []
    for j in np.argsort(feature_activations[:, i])[::-1][:10]:
      if feature_activations[j, i] > 0.1:
        expansions_map[feature_labels[i]].append(ind_to_question[j])
  elif i in feature_labels and feature_labels[i] in texts and nonzero_features[i] > 5:
    num_index = texts.index(feature_labels[i])
    numbers[num_index] = max(numbers[num_index], nonzero_features[i])



In [100]:
len(texts)

16894

In [101]:

# -----------------------------------------------------------------------------
# 1. Sample data
# -----------------------------------------------------------------------------
# texts = [
#     "Apple is looking at buying U.K. startup for $1 billion",
#     "Apple pie recipe made with fresh ingredients",
#     "The stock market is volatile today",
#     "Bananas are my favorite fruit",
#     "Microsoft announced new Surface devices",
#     "Google plans to expand into consumer banking",
#     "I love banana smoothies",
#     "Investors are concerned about inflation",
# ]
# numbers = [50, 30, 20, 15, 35, 40, 25, 10]

# (You would replace the above with your actual list of (number, text) pairs.)

# -----------------------------------------------------------------------------
# 2. Compute embeddings for semantic similarity
#    We'll use a small model from sentence-transformers for demonstration.
# -----------------------------------------------------------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')  # or any other you prefer
embeddings = model.encode(texts)

print("Encoded!")
# -----------------------------------------------------------------------------
# 3. Reduce dimensionality to 2D (using UMAP here, could use t-SNE/PCA)
# -----------------------------------------------------------------------------
reducer = umap.UMAP(n_neighbors=3, n_components=2, random_state=42)
coords_2d = reducer.fit_transform(embeddings)

# Extract X, Y coordinates
x_coords = coords_2d[:, 0]
y_coords = coords_2d[:, 1]

# -----------------------------------------------------------------------------
# 4. Prepare data for Bokeh
#    - Circle size proportional to numbers
#    - Truncate text for display if needed (here: up to 20 chars)
# -----------------------------------------------------------------------------
circle_sizes = [n/2 for n in numbers]  # Adjust scaling factor as needed
truncated_texts = [t[:20] + "..." if len(t) > 20 else t for t in texts]

expansions_list = []
for t in texts:
    expansions_list.append(expansions_map.get(t, []))

full_source = ColumnDataSource(data=dict(
    x=x_coords,
    y=y_coords,
    text=texts,
    truncated=truncated_texts,
    number=numbers,
    size=circle_sizes,
    expansions=expansions_list,
))

source = ColumnDataSource(data=full_source.data.copy())

div = Div(
    text="<i>Click a bubble to see details...</i>",
    width=400,
    height=200
)
# -----------------------------------------------------------------------------
# 5) Define a callback on selection
# -----------------------------------------------------------------------------
def callback(attr, old, new):
    """
    Whenever the user taps a circle, we get the index of that circle
    and look up expansions in expansions_map. We then display them in 'div'.
    """
    selected_indices = source.selected.indices
    if not selected_indices:
        # Nothing selected
        div.text = "<i>Click a bubble to see details...</i>"
        return

    # We only allow single selection for simplicity
    idx = selected_indices[0]

    # The "key" we use is the original text to map expansions
    key = source.data["text"][idx]

    # Look up expansions in your dictionary
    expansions = expansions_map.get(key, [])
    if not expansions:
        div.text = f"<b>No expansions found for:</b> {key}"
    else:
        # Format expansions as a bullet list
        expansions_html = "".join(f"<li>{e}</li>" for e in expansions)
        div.text = f"<h3>{key}</h3><ul>{expansions_html}</ul>"

# Attach callback to the "selected" property of the data source
callback_js = CustomJS(args=dict(source=source, div=div), code="""
    const inds = source.selected.indices;
    if (inds.length == 0) {
        div.text = "<i>Click a bubble to see details...</i>"
        return;
    }
    const i = inds[0];
    const expansions = source.data['expansions'][i]; // an array of strings, for example
    const text = source.data['text'][i];
    let expansions_html = "";
    for (let e of expansions) {
        expansions_html += `<li>${e}</li>`;
    }
    div.text = `<h3>${text}</h3><ul>${expansions_html}</ul>`;
""");
source.selected.js_on_change("indices", callback_js);

Encoded!


/Users/nickjiang/anaconda3/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [102]:
torch.save({
  "coords_2d": coords_2d,
}, "./feature_viz/coords_2d.pt")

In [88]:
# -----------------------------------------------------------------------------
# 5. Create Bokeh plot
# -----------------------------------------------------------------------------
p = figure(
    title="Text Bubbles by Semantic Similarity",
    width=1000,
    height=1000,
    tools="pan,wheel_zoom,reset,save",
    active_scroll="wheel_zoom"
)

# Draw circles
p.circle(
    x='x', y='y',
    size='size',
    source=source,
    fill_alpha=0.6,
    line_color='black'
)

# Add a hover tool that shows full text and number
hover = HoverTool(
    tooltips=[
        ("Text", "@text"),
        ("Number", "@number"),
    ]
)
p.add_tools(hover)

# Optionally add visible labels (which may overlap if points are close).
# To reduce label overlap, you could set x_offset/y_offset or rely on hover.
labels = LabelSet(
    x='x', y='y',
    level='glyph',
    x_offset=5, y_offset=5,  # small offset so label won't be exactly on the circle
    source=source,
)
p.add_layout(labels)
# ------------------------------------------------------------------------------
# CustomJS Callback: Toggle label visibility based on zoom level
# ------------------------------------------------------------------------------
threshold_x = 5
threshold_y = 5

code = """
// Get the current range size
const range_width = p.x_range.end - p.x_range.start
const range_height = p.y_range.end - p.y_range.start

// Toggle label visibility
if (range_width < threshold_x && range_height < threshold_y) {
    labelset.visible = true
} else {
    labelset.visible = false
}
"""

callback = CustomJS(
    args=dict(
        p=p,
        labelset=labels,
        threshold_x=threshold_x,
        threshold_y=threshold_y,
    ),
    code=code
)

# Fire callback whenever x_range or y_range changes
p.x_range.js_on_change("start", callback)
p.x_range.js_on_change("end", callback)
p.y_range.js_on_change("start", callback)
p.y_range.js_on_change("end", callback)

# --------------------------------------------------------------------------------
# 4. Add a Slider to filter data
#    - If nonZero < slider value, hide the point.
# --------------------------------------------------------------------------------
min_nonzero = 5
max_nonzero = 960

slider = Slider(
    start=min_nonzero,  # or start=0 if you prefer
    end=max_nonzero,
    value=min_nonzero,  # initial value
    step=1,
    title="Minimum Non-Zero Features"
)

# JavaScript code for filtering
callback_code = """
// 'cb_obj.value' is the slider's current value
var threshold = cb_obj.value;

// Access the two data sources
var full_data = full_source.data;
var s_data = source.data;

// We'll build new arrays for only the points that meet the threshold
var newX = [];
var newY = [];
var newText = [];
var newTruncated = [];
var newNumber = [];
var newSize = [];

for (var i = 0; i < full_data['x'].length; i++) {
    if (full_data['number'][i] >= threshold) {
        newX.push(full_data['x'][i]);
        newY.push(full_data['y'][i]);
        newText.push(full_data['text'][i]);
        newTruncated.push(full_data['truncated'][i]);
        newNumber.push(full_data['number'][i]);
        newSize.push(full_data['size'][i]);
    }
}

// Update source with these filtered arrays
s_data['x'] = newX;
s_data['y'] = newY;
s_data['text'] = newText;
s_data['truncated'] = newTruncated;
s_data['number'] = newNumber;
s_data['size'] = newSize;

// Emit change so Bokeh knows to re-render
source.change.emit();
"""

# Create the callback
slider_callback = CustomJS(
    args=dict(source=source, full_source=full_source),
    code=callback_code
)

# When the slider's 'value' changes, run the callback
slider.js_on_change("value", slider_callback)

# --------------------------------------------------------------------------------
# 5. Layout: put the slider above or beside the plot
# --------------------------------------------------------------------------------
layout = row(column(slider, p), div)
curdoc().add_root(layout)
curdoc().title = "Bubble Click Demo"

show(layout)

In [93]:
torch.save({
  "coords_2d": coords_2d,
}, "./feature_viz/coords_2d.pt")

In [73]:
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, HoverTool, LabelSet, Div
from bokeh.plotting import figure
from bokeh.layouts import column, row

# -----------------------------------------------------------------------------
# 1) Your (number, text) pairs plus expansions
# -----------------------------------------------------------------------------
texts = [
    "Apple is looking at buying U.K. startup for $1 billion",
    "Apple pie recipe made with fresh ingredients",
    "The stock market is volatile today",
    "Bananas are my favorite fruit",
]
numbers = [50, 30, 20, 10]

# This dictionary maps an original text to a list of expansions/child texts.
# For example:
expansions_map = {
    "Apple is looking at buying U.K. startup for $1 billion": [
        "First rumored in the Wall Street Journal",
        "CEO Tim Cook rumored to sign the deal next week",
        "Regulatory approval pending",
    ],
    "Apple pie recipe made with fresh ingredients": [
        "Use Granny Smith apples",
        "Add cinnamon and sugar",
        "Bake for 45 minutes",
    ],
    "The stock market is volatile today": [
        "NASDAQ down 1.2%",
        "S&P hits record trading volume",
    ],
    "Bananas are my favorite fruit": [
        "High in potassium",
        "Great in smoothies or bread",
    ]
}

# -----------------------------------------------------------------------------
# 2) Prepare the Bokeh data source
#    We'll store each point's expansions-key (the original text) so we can look up later.
# -----------------------------------------------------------------------------
import numpy as np
x = np.array([10, 20, 30, 40])
y = np.array([15, 25, 10, 5])
circle_sizes = [n/2 for n in numbers]

source = ColumnDataSource(data=dict(
    x=x,
    y=y,
    text=texts,
    size=circle_sizes,
))

# -----------------------------------------------------------------------------
# 3) Create the figure
# -----------------------------------------------------------------------------
p = figure(
    title="Click Bubbles to See Expansions",
    width=600,
    height=400,
    tools="pan,wheel_zoom,box_zoom,reset,tap"  # tap tool needed for clicks
)

# Draw circles
p.circle(
    x="x", y="y",
    size="size",
    source=source,
    fill_alpha=0.6,
    line_color="black",
    selection_color="red"  # highlight a selected circle in red
)

# Hover for quick info
hover = HoverTool(tooltips=[("Text", "@text"), ("Number", "@size")])
p.add_tools(hover)

# Optional: Add a LabelSet if you want small labels on the plot
labels = LabelSet(
    x="x",
    y="y",
    text="text",
    x_offset=5,
    y_offset=5,
    source=source,
    visible=False,  # hide by default if you want
)
p.add_layout(labels)

# -----------------------------------------------------------------------------
# 4) A Div to show expansions
# -----------------------------------------------------------------------------
div = Div(
    text="<i>Click a bubble to see details...</i>",
    width=400,
    height=200
)

# -----------------------------------------------------------------------------
# 5) Define a callback on selection
# -----------------------------------------------------------------------------
def callback(attr, old, new):
    """
    Whenever the user taps a circle, we get the index of that circle
    and look up expansions in expansions_map. We then display them in 'div'.
    """
    selected_indices = source.selected.indices
    if not selected_indices:
        # Nothing selected
        div.text = "<i>Click a bubble to see details...</i>"
        return

    # We only allow single selection for simplicity
    idx = selected_indices[0]

    # The "key" we use is the original text to map expansions
    key = source.data["text"][idx]

    # Look up expansions in your dictionary
    expansions = expansions_map.get(key, [])
    if not expansions:
        div.text = f"<b>No expansions found for:</b> {key}"
    else:
        # Format expansions as a bullet list
        expansions_html = "".join(f"<li>{e}</li>" for e in expansions)
        div.text = f"<h3>{key}</h3><ul>{expansions_html}</ul>"

# Attach callback to the "selected" property of the data source
callback_js = CustomJS(args=dict(source=source, div=div), code="""
    const inds = source.selected.indices;
    if (inds.length == 0) {
        div.text = "<i>Click a bubble to see details...</i>"
        return;
    }
    const i = inds[0];
    const expansions = source.data['expansions'][i]; // an array of strings, for example
    const text = source.data['text'][i];
    let expansions_html = "";
    for (let e of expansions) {
        expansions_html += `<li>${e}</li>`;
    }
    div.text = `<h3>${text}</h3><ul>${expansions_html}</ul>`;
""");
source.selected.js_on_change("indices", callback_js);


# -----------------------------------------------------------------------------
# 6) Layout
# -----------------------------------------------------------------------------
layout = row(p, div)
curdoc().add_root(layout)
curdoc().title = "Bubble Click Demo"

show(layout)